# Strands를 사용한 임상 증거 연구 에이전트 - 외부 & 내부 도구 사용
이 노트북에서는 오픈소스 Strands 에이전트 프레임워크를 사용하여 임상 증거 연구 에이전트를 생성합니다

#### Strands 에이전트 및 필수 종속성 설치

In [ ]:
%pip install strands-agents strands-agents-tools==0.2.9 xmltodict --quiet

#### 최신 버전의 boto3가 아래에 표시되는지 확인
아래에 출력된 boto3 버전이 **1.37.1** 이상인지 확인하세요.

In [ ]:
%pip show boto3

#### 필수 라이브러리 가져오기

In [ ]:
import os
import boto3
import json
import uuid
import requests
from pathlib import Path
from typing import Dict, Any, List
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import retrieve

from utils.PubMed import PubMed

# KB 도구 변수 초기화
kb_tool = None

# AWS 계정 정보 확인
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

#### documents/ 디렉토리의 문서를 S3 버켓에 업로드

In [ ]:
# S3 클라이언트 초기화
s3_client = boto3.client('s3')
environment_name = 'env1'
bucket_name = f'clinical-kb-{environment_name}-{account_id}-{region}'

documents_dir = Path('./documents')
if documents_dir.exists():
    for file_path in documents_dir.rglob('*'):
        if file_path.is_file():
            s3_key = str(file_path.relative_to('.'))
            try:
                s3_client.upload_file(str(file_path), bucket_name, s3_key)
                print(f'업로드 완료: {s3_key} -> s3://{bucket_name}/{s3_key}')
            except Exception as e:
                print(f'업로드 실패: {s3_key} - {str(e)}')
else:
    print('documents/ 디렉토리가 존재하지 않습니다.')

#### AWS 클라이언트 설정
도구에서 사용할 AWS 서비스 클라이언트를 정의합니다.

In [ ]:
# AWS 클라이언트 초기화
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
bedrock_agent_client = boto3.client("bedrock-agent", region_name=region)

print(f"리전: {region}")
print(f"계정 ID: {account_id}")

#### 내부 문서 검색을 위한 지식 베이스 설정

이 예제에서는 Strands 에이전트의 내장 도구인 `retrieve`를 사용할 것입니다. 이 도구는 RAG, 메모리 및 기타 목적을 위해 Amazon Bedrock 지식베이스에서 데이터를 의미론적으로 검색합니다. 이 도구는 지식베이스 ID를 필요로 하며, 이는 아래에 정의된 환경 변수 `KNOWLEDGE_BASE_ID`를 통해 제공될 것입니다.

In [ ]:
# 지식 기반 찾기
response = bedrock_agent_client.list_knowledge_bases()

# 지식 기반 탐색
ncbi_kb_id = None
for kb in response['knowledgeBaseSummaries']:
    kb_name = kb['name']
    if 'ncbiKnowledgebase' in kb_name:
        ncbi_kb_id = kb['knowledgeBaseId']
        break

if ncbi_kb_id:
    print(f"Found Knowledge Base ID: {ncbi_kb_id}")
    os.environ["KNOWLEDGE_BASE_ID"] = ncbi_kb_id
    print("Knowledge Base will be integrated using direct Strands tool approach")
else:
    print("Warning: Knowledge Base not found. Internal evidence retrieval may not work.")
    ncbi_kb_id = None

# Strands 에이전트 생성
이 섹션에서는 Strands 프레임워크를 사용하여 에이전트를 생성합니다

#### 에이전트 구성 및 지침 정의

In [ ]:
clinical_research_agent_name = "Clinical-evidence-researcher-strands"
clinical_research_agent_description = "Research internal and external evidence using Strands framework"
clinical_research_agent_instruction = """당신은 HER2 바이오마커와 관련된 내부 및 외부 증거를 요약하는 데 특화된 의료 연구 보조 AI입니다. 
주요 임무는 사용자 질의를 해석하고, 내부 및 외부 증거를 수집하며, 그 결과에 기반한 관련 의학적 통찰력을 제공하는 것입니다. 
특정 질문에 필요한 적절한 도구만 사용하십시오. 내부 증거 검색 시 항상 먼저 지식 기반 검색 도구를 사용하십시오. 다음 지침을 주의 깊게 따르십시오: 

1. retreive 도구 사용시: 
   a. 내부 증거의 경우 지식 기반을 활용하여 관련 정보를 검색하십시오. 
   b. 응답 시 항상 특정 콘텐츠 조각에 대한 인용(예. s3://bucket_name/test.pdf)을 포함하십시오. retreive 도구의 응답에서 location 정보를 활용하세요.

2. PubMed 쿼리 시:
   a. 각 관련 연구의 결과를 요약하고 해당 연구의 특정 PubMed 웹 링크를 인용하십시오.
   b. JSON 출력에는 '링크', '제목', '요약'이 포함됩니다. 
   c. 응답 시 항상 각 연구의 제목과 링크(예: 'https://pubmed.ncbi.nlm.nih.gov/')를 포함하십시오.  

4. 응답 제공 시: 
   a. 사용자의 질의에 대한 이해를 간략히 요약하여 시작하십시오.  
   b. 질의 해결을 위해 취하는 단계를 설명하십시오. 필요한 경우 사용자에게 추가 설명을 요청하십시오.  
   c. 내부 증거(지식 기반)와 외부 증거(PubMed API)에서 생성된 응답을 구분하십시오.  
   d. 발견 사항과 의학 연구에 대한 잠재적 함의를 간결히 요약하여 결론지으십시오.
"""

#### Strands 에이전트용 도구 정의
PubMed를 쿼리하기 위해 커스텀 도구를 사용하며, Strands 프레임워크의 retrieve 도구와 결합할 예정입니다. retrieve 도구는 커스텀 도구처럼 함수로 정의할 필요가 없으므로, 단순히 도구 목록에 추가하면 됩니다.

In [ ]:
# Strands @tool 데코레이터를 사용하여 도구 정의
@tool
def query_pubmed(query: str) -> str:
    """
    사용자의 질의에 기반하여 PubMed에서 관련 생의학 문헌을 검색합니다.
    이 도구는 PubMed 초록을 검색하여 제목, 링크 및 요약이 포함된 관련 연구를 반환합니다.
    
    Args:
        query (str): PubMed 검색어
    
    Returns:
        str: PubMed 검색 결과(제목, 링크, 요약 포함)를 포함한 JSON 문자열
    """
    
    pubmed = PubMed()

    print(f"\nPubMed 쿼리: {query}\n")
    result = pubmed.run(query)
    print(f"\nPubMed 결과: {result}\n")
    return result

# 사용자 정의 도구 목록 생성
clinical_research_agent_tools = [query_pubmed, retrieve]
print(f"Strands 에이전트를 위한 {len(clinical_research_agent_tools)} 개의 사용자 정의 도구 생성됨")

#### Strands용 AWS Bedrock 프로바이더 설정

In [ ]:
# Strands용 베드락 모델 생성
model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    region_name=region,
    temperature=0.1,
    streaming=False
)

#### Strands 에이전트 생성

In [ ]:
# Strands 에이전트 생성
try:
    # 맞춤형 도구 사용
    clinical_evidence_agent = Agent(
        model=model,
        tools=clinical_research_agent_tools,
        system_prompt=clinical_research_agent_instruction
    )
    
    print(f"Strands 에이전트 생성 성공: {clinical_research_agent_name}")
    print(f"에이전트에 {len(clinical_research_agent_tools)}개의 도구가 사용 가능합니다:")
    for tool in clinical_research_agent_tools:
        print(f"  - {tool.__name__}")
    
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

#### Strands 에이전트 테스트

In [ ]:
# 연구 질의로 에이전트 테스트
test_query = "HER2 양성 유방암에서 HER2 표적 치료제의 효과에 관한 증거를 내부 지식베이스와 PubMed에서 검색해 주실 수 있나요?"

print(f"쿼리를 사용한 테스트 에이전트: {test_query}")
print("=" * 140)

try:
    # 에이전트 실행
    response = clinical_evidence_agent(test_query)
    
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

#### 고급 사용 예제

In [ ]:
# 더 복잡한 쿼리의 예시
complex_queries = [
    "HER2 양성 유방암에서 트라스투주맙 내성 메커니즘에 대한 증거 검색",
    "HER2 표적 치료제와 면역치료 병용요법의 효과에 관한 연구 찾기",
    "HER2 바이오마커와 치료 반응 예측에 대해 내부 지식 기반은 무엇을 말하는가?"
]

def test_complex_query(query: str):
    """
    Test a complex query with the agent
    """
    print(f"\n테스트 쿼리: {query}")
    print("-" * 100)
    
    try:
        response = clinical_evidence_agent(query)
    except Exception as e:
        print(f"Error: {e}")

for query in complex_queries: 
    test_complex_query(query)

## 요약
이 노트북은 Strands 프레임워크를 사용하여 Bedrock Knowledge 및 PubMed API에 연결하는 에이전트를 구축하는 방법을 보여줍니다.

### 사용 가능한 도구:
- `query_pubmed`: PubMed에서 의학 문헌 검색
- `KnowledgeBase`: NCBI 지식베이스에서 내부 증거 검색

### 연구 기능:
- PubMed API 통합을 통한 **의학 문헌 검색**
- 큐레이션된 지식베이스로부터의 **내부 증거 검색**
- 다중 출처 통합을 통한 **종합적 연구 종합**
- 원본 연구 링크를 포함한 **인용 추적**
- HER2 양성 유방암 연구를 위한 **바이오마커 연구** 특화 기능